# Advanced ML: Домашнее задание 4

Четвёртое домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 
В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена, которые выглядели примерно так:

![](./dancing_men.png)

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.
В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:

[corpora.zip](./corpora.zip)

In [1]:
import zipfile
import random
from collections import Counter
import numpy as np
from tqdm import tqdm, trange
from time import sleep

In [2]:
alphabet = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')
def read_file_from_archive(archive, name):
    return archive.read(name).decode("utf-8").replace('\t', ' ')

def read_files():
    archive = zipfile.ZipFile('corpora.zip', 'r')
    str_anna = read_file_from_archive(archive, "AnnaKarenina.txt")
    str_war = read_file_from_archive(archive, "WarAndPeace.txt")
    str_war_eng = read_file_from_archive(archive, "WarAndPeaceEng.txt")
    return str_anna, str_war, str_war_eng

def filter_text(text, alphabet):
    return ' '.join(''.join([ch if ch in alphabet else ' ' for ch in text.lower()]).split())

def get_ngram_count(text, n):
    return Counter([text[i:i+n] for i in range(len(text) - n + 1)])

def calc_frequenties(text, n_gram):
    freq_dict = get_ngram_count(text, n_gram)
    sum_freqs = sum(freq_dict.values())
    return Counter({k: v / sum_freqs for k, v in freq_dict.items()})

def get_cipher_alphabet(alphabet):
    cipher_alphabet = [ord(x) for x in alphabet]
    cipher_alphabet = [chr(x + 500) for x in cipher_alphabet]
#     random.Random(42).shuffle(cipher_alphabet) 
    np.random.shuffle(cipher_alphabet) 
    return cipher_alphabet

def get_cipher_dict(alphabet):
    cipher_dict = dict.fromkeys(alphabet, '')
    cipher_alphabet = get_cipher_alphabet(alphabet)
    for i in range(len(alphabet)):
        cipher_dict[alphabet[i]] = cipher_alphabet[i]
    return cipher_dict

def get_map_dict(freques_alphabet, freques_text):
    alphabet_freq = [x[0] for x in freques_alphabet.most_common()]
    text_freq = [x[0] for x in freques_text.most_common()]
    return dict(zip(text_freq, alphabet_freq))
        
def cipher(text, cipher_dict):
    return ''.join(cipher_dict[ch] for ch in text if ch in cipher_dict)

def replace_ngram(text, map_dict, n_gram):
    null_ch = chr(0)
    replaced = [null_ch for i in range(len(text))]
    for k, v in map_dict.items():
        i_s = text.find(k)
        while i_s != -1:
            for i in range(n_gram):
                if replaced[i_s + i] == null_ch:
                    replaced[i_s + i] = v[i]
            i_s = text.find(k, i_s + 1)
    return ''.join(replaced)

def get_letter_accuracy(orig_text, deciphered_text):
    good = 0
    for i in range(len(orig_text)):
        if orig_text[i] == deciphered_text[i]:
            good +=1
    return good / len(orig_text)

def freque_decipher(etalon_text, ciphered_text, n_gram):
    freques_alphabet = calc_frequenties(etalon_text, n_gram)
    freques_text = calc_frequenties(ciphered_text, n_gram)
    decipher_dict = get_map_dict(freques_alphabet, freques_text)
    return replace_ngram(ciphered_text, decipher_dict, n_gram)

In [3]:
str_anna, str_war, str_war_eng = read_files()
rus_text = filter_text(str_anna + str_war, alphabet)

## 1. Реализуйте базовый частотный метод по Шерлоку Холмсу:

* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе совсем вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.

In [4]:
"""Подсчёт частот по корпусам"""
calc_frequenties(rus_text, 1)

Counter({'а': 0.06940753885954973,
         'н': 0.056983173333903486,
         ' ': 0.16590579632678368,
         'к': 0.028987192918688932,
         'р': 0.034576553847469584,
         'е': 0.07105642385238717,
         'и': 0.05546684911590515,
         'о': 0.09565371704688803,
         'д': 0.02480981804964615,
         'з': 0.013992858822774797,
         'с': 0.04415214556028308,
         'м': 0.024146158944645186,
         'ы': 0.01558529858245494,
         'х': 0.00666481366658827,
         'т': 0.04928609608518099,
         'в': 0.03907806119176413,
         'л': 0.04198798400718394,
         'ь': 0.0163994783092087,
         'г': 0.01576617989779992,
         'ч': 0.01334459387227128,
         'я': 0.0183536806995788,
         'ш': 0.007337025079643369,
         'й': 0.009010711765837806,
         'ф': 0.0012785700540933484,
         'п': 0.020499882405764256,
         'ж': 0.009185178850142183,
         'у': 0.022912916123238758,
         'э': 0.0028427871971948428,
        

In [5]:
"""Лукъяненко, Линия грёз"""
sample_text_raw = '''
Больше всего Кей не любил детей. Сказалось ли на этом его собственное детство – в приюте «Новое поколение» на Альтосе, неизвестно. Как бы там ни было, он никогда не задерживался на одной планете больше девяти месяцев. На тех планетах, которые во время Смутной Войны прошли фертильную обработку, и честно служили поставщиками пушечного мяса для Империи, он не задерживался более четырех с половиной месяцев.
Кроме этого Кей не любил, когда его убивали. Порой это было крайне болезненно, и всегда – связано с немалыми тратами. А деньги Кею были нужны. Он любил свой гиперкатер – требующий дорогостоящего ухода, женщин – не требующих столь многого, вина Империи и Мршанской ассоциации, запахи работы старых Клаконских мастеров и те удовольствия прочих рас, которые способен понять и выдержать человек.
Сейчас две его антипатии сложились воедино. И самым неприятным было не то, что его собирался убить ребенок, из‑за ребенка, и одним из самых неприятных способов. Беда была в том, что Кей не успел оплатить продление аТана.
'''
sample_text = filter_text(sample_text_raw, alphabet) # # Очищенный текст примера
ciphered_sample_text = cipher(sample_text, get_cipher_dict(alphabet)) # Зашифрованный текст примера

In [6]:
deciphered_text = freque_decipher(rus_text, ciphered_sample_text, 1)
print('Оригинальный текст\n\n', sample_text, "\n\n")
print('Зашифрованный текст\n\n', ciphered_sample_text, "\n\n")
print('Расшифрованный текст\n\n', deciphered_text, "\n\n\n")
print('Доля правильно расшифрованных букв', get_letter_accuracy(sample_text, deciphered_text))

Оригинальный текст

 больше всего кей не любил детей сказалось ли на этом его собственное детство в приюте новое поколение на альтосе неизвестно как бы там ни было он никогда не задерживался на одной планете больше девяти месяцев на тех планетах которые во время смутной войны прошли фертильную обработку и честно служили поставщиками пушечного мяса для империи он не задерживался более четырех с половиной месяцев кроме этого кей не любил когда его убивали порой это было крайне болезненно и всегда связано с немалыми тратами а деньги кею были нужны он любил свой гиперкатер требующий дорогостоящего ухода женщин не требующих столь многого вина империи и мршанской ассоциации запахи работы старых клаконских мастеров и те удовольствия прочих рас которые способен понять и выдержать человек сейчас две его антипатии сложились воедино и самым неприятным было не то что его собирался убить ребенок из за ребенка и одним из самых неприятных способов беда была в том что кей не успел оплатить продление а

## 2. Биграммы

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм.

In [7]:
deciphered_text_bi = freque_decipher(rus_text, ciphered_sample_text, 2)
print('Расшифрованный текст\n\n', deciphered_text_bi, "\n\n\n")
print('Доля правильно расшифрованных букв', get_letter_accuracy(sample_text, deciphered_text_bi))

Расшифрованный текст

 кпо ао  вояа исне о заотркавнин с еол  чрали   вытоолояа сме к ал по аве кла   ор ено  плао одмпоално   вл  точо  нми р к а ие нем трое и емйа нее нм оы е о осоносетл оле  внн пн овв нно епо ао авреаи ежодотее  втиг овв норг и тотяо  а  завл слбо пна абд  ороули кноапу лнуенен коин си тж к а спсспри од кршас ели осавд пяа езв варлстлено и нее о осоносетл олнепоыо ттедзаг с опо    пнаежодоте ироно ытояа исне о заотр и оы вояа зоттл и одр наыта емйа ин д о епоы  нл а и  воны  с яос а се нвл ери тн орли  вааледи иаинемли  яад енетзаотр сланаднено тнно тзаснуаснааирвя  точаояа зхиы вшаласае о тзаснуасг стоорае пя яа   т втлено и и ехсвасм наибомкускли ос зни н кои  скраяг ивем ес егоео кнооеси то зтвлпо  келл орокегон ю и тотяо с дчмеал одеесрси  воноитаратмй  р  сскюоюка о ояа ваакктаки сй спродрь а х  а и силеое нер еогоонемйа  о та тта ояа смотнл олизоторазаса п стиео взасал  ви ннаности силсге нер еогог с дчмкоенеоы вем  в  тоолтта исне о зоегченмвтакор оронбално

## 3. MCMC

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:

* предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
* реализуйте и протестируйте его, убедитесь, что результаты улучшились.


Будем считать что текст представляет собой марковскую цепь, то есть следующий символ зависит от предыдущих. 

* Инициализируем перестановку маппинга отдельных букв частотным способом, над этим маппингом и будем работать
* На каждом шаге алгоритма будем считать правдподобие данной перестановки в зависимости от заданного числа N-грамм. (Для этого придётся применять маппинг на зашифрованный текст и заново для него считать частоты N-грамм)
* Если правдоподобие перестановки больше, текущее, принимаем её. В противном случае, принимаем её, с вероятностью new_likelihood/old_likelihood)
* Выдаём лучшую найденную перестановку по правдоподобию

In [8]:
class MCMCModel:
    def __init__(self, etalon_text, n_gram = 2, n_iter = 50000):
        self.unifreqs = calc_frequenties(etalon_text, 1)
        self.freqs = calc_frequenties(etalon_text, n_gram) if n_gram > 1 else self.unifreqs
        self.n_gram = n_gram
        self.n_iter = n_iter     
        pass 
        
    def log_likelihood(self, text):
        ngram_counter = get_ngram_count(text, self.n_gram)
        none_freq = min(self.freqs.values()) / 2
        result = 0
        for ngram, count in ngram_counter.items():
            freq = self.freqs.get(ngram)
            if freq is None:
                freq = none_freq
                none_freq /= 2 
            result += count * np.log(freq)
        return result
    
    def get_random_permute_2_symbols_dict(self, decipher_dict):
        perm_decipher_dict = decipher_dict.copy()
        k1, k2 = np.random.choice(list(perm_decipher_dict.keys()), 2, replace=False)
        perm_decipher_dict[k1], perm_decipher_dict[k2] = perm_decipher_dict[k2], perm_decipher_dict[k1]
        return perm_decipher_dict
   
    def fit(self, ciphered_text):
        decipher_dict = get_map_dict(self.unifreqs, calc_frequenties(ciphered_text, 1))
        best_decipher_dict = decipher_dict
        max_llh = self.log_likelihood(cipher(ciphered_text, decipher_dict))
        cur_llh = max_llh
        for i in trange(self.n_iter):
            new_decipher_dict = self.get_random_permute_2_symbols_dict(decipher_dict)
            new_text = cipher(ciphered_text, new_decipher_dict)
            new_llh = self.log_likelihood(new_text)
            if new_llh > cur_llh or np.random.rand() < np.exp(new_llh - cur_llh):
                decipher_dict = new_decipher_dict
                cur_llh = new_llh
                if cur_llh > max_llh:
                    max_llh = cur_llh
                    best_decipher_dict = decipher_dict
        self.decipher_dict = best_decipher_dict
        return self
    
    def predict(self, ciphered_text):
        return cipher(ciphered_text, self.decipher_dict)

In [9]:
model = MCMCModel(rus_text, n_gram=2, n_iter=20000)
deciphered_text_mcmc = model.fit(ciphered_sample_text).predict(ciphered_sample_text)
print("\n", deciphered_text_mcmc, "\n")
print('Доля правильно расшифрованных букв', get_letter_accuracy(sample_text, deciphered_text_mcmc))

100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [00:15<00:00, 1330.02it/s]


 больше всего кей не любил детей сказалось ли на этом его собственное детство в приюте новое поколение на альтосе неизвестно как бы там ни было он никогда не задерживался на одной планете больше девяти месяцев на тех планетах которые во время смутной войны прошли фертильную обработку и честно служили поставщиками пушечного мяса для империи он не задерживался более четырех с половиной месяцев кроме этого кей не любил когда его убивали порой это было крайне болезненно и всегда связано с немалыми тратами а деньги кею были нужны он любил свой гиперкатер требующий дорогостоящего ухода женщин не требующих столь многого вина империи и мршанской ассоциации запахи работы старых клаконских мастеров и те удовольствия прочих рас которые способен понять и выдержать человек сейчас две его антипатии сложились воедино и самым неприятным было не то что его собирался убить ребенок из за ребенка и одним из самых неприятных способов беда была в том что кей не успел оплатить продление атана 

Доля правиль

## 4. Расшифруйте сообщение:

```
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
```
Или это (они одинаковые, но сообщали о проблемах с юникодом):
```
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ
```

In [10]:
task_text = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"
model = MCMCModel(rus_text, n_gram=3, n_iter=20000)
deciphered_task_text_mcmc = model.fit(task_text).predict(task_text)
print("\n", deciphered_task_text_mcmc, "\n")

100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [00:09<00:00, 2043.48it/s]


 если вы видите нормальный или почти нормальный текст у штого сообжения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обежаю 



На коротких текстах вроде текста задания может потребоваться несколько запусков для получения внятного результата, так как функция, бывает, прыгает в локальные максимумы и уже не выбирается из них.

## 5. Бонус

> а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно

Код задания написан таким образом, что переход к ngram'ам делается простым изменением параметра n_gram. Кстати, ответ на задание был получен именно при `n_gram=3`, при этом значении расшифровать ответ получается чаще, чем при 2.

Для примера - расчёт на n_grams=4 (гораздо больше локальных максимумов, требуется много перезапусков)

2ю часть бонуса с генерацией множества тестовых перестановок не выполнял.

In [14]:
task_text = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"
model = MCMCModel(rus_text, n_gram=4, n_iter=20000)
deciphered_task_text_mcmc = model.fit(task_text).predict(task_text)
print("\n", deciphered_task_text_mcmc, "\n")

100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [00:21<00:00, 937.43it/s]


 если вы видите нормальный или почти нормальный текст у штого сообжения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обежаю 



## 6. Бонус
>какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

Например, можно использовать эту модель для (полу)автоматического определения кодировки текстового файла(предварительно, конечно, придётся для такой программы создать наборы частот для каждого поддерживаемого языка)